In [1]:
# Import packages used in this notebook
import os, sys, re
import pandas as pd
import numpy as np

In [3]:
# Load the name mapping for the column names
# sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [74]:
dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-1/Data/CTD/'
#datapath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/Data/CTD/'

In [75]:
for file in os.listdir(dirpath):
    if '.btl' in file:
        print(file)

KN22104001.btl
KN22104002.btl
KN22104003.btl
KN22104004.btl
KN22104005.btl
KN22104006.btl
KN22104007.btl
KN22104009.btl
kn22104999.btl


In [80]:
# dirpath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/Data/CTD/'
dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-1/Data/CTD/'

In [81]:
# Now write a function to autopopulate the bottle summary sample sheet
sbe_name_map;
datapath;

# Now 
files = [x for x in os.listdir(dirpath) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(dirpath+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
    
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
    
    # Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
            
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)
        
    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast'] = cast
    
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(dirpath+outname)

In [82]:
df

,Bottle Position,Date Time,PrDM,DepSM,Latitude,Longitude,T090C,T190C,C0S/m,C1S/m,...,Sal11,Sbeox0ML/L,OxsolML/L,CStarAt0,CStarTr0,Hex name,Start Latitude,Start Longitude,Start Time,Cast
0,2,Sep 05 2014 14:29:00,0.174,0.172,64.15140,-21.93566,15.6976,15.5727,0.000391,0.000191,...,0.0083,6.9827,6.95106,0.4873,88.5294 (avg),C:\Data\ctd\decktest_2014_Sept_05.hex,64 09.08 N,021 56.13 W,2014-09-05T14:26:29Z,999
1,4,Sep 05 2014 14:29:10,0.162,0.160,64.15140,-21.93566,15.7024,15.5744,0.000391,0.000190,...,0.0083,6.9818,6.95034,0.4873,88.5294 (avg),C:\Data\ctd\decktest_2014_Sept_05.hex,64 09.08 N,021 56.13 W,2014-09-05T14:26:29Z,999


In [25]:
filename[filename.index('.')-3:filename.index('.')]

'023'

In [55]:
df = pd.read_csv(dirpath+'ar30-03006.sum')

In [57]:
df.columns.values

array(['Unnamed: 0', 'Bottles', 'Fired,"Pressure,', 'Digiquartz',
       '[db]","Temperature', '[ITS-90,', 'deg', 'C]","Temperature,', '2',
       'C]",Conductivity', '[S/m],"Conductivity,', '[S/m]","Oxygen',
       'raw,', 'SBE', '43', '[V]","Fluorescence,', 'WET', 'Labs',
       'ECO-AFL/FL', '[mg/m^3]","Turbidity,', 'ECO', '[NTU]","Salinity,',
       'Practical', '[PSU]",SPAR/Surface', 'Irradiance,Scan',
       'Count,Flag,Datetime,Cruise', 'ID,Cast', 'Cast'], dtype=object)

In [50]:
df.shape

(19, 23)

In [83]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(dirpath):
    if '.sum' in file:
        df = df.append(pd.read_csv(dirpath+file))
    else:
        pass
            
        

In [84]:
df

,Unnamed: 0,Bottle Position,Date Time,PrDM,DepSM,Latitude,Longitude,T090C,T190C,C0S/m,...,Sal11,Sbeox0ML/L,OxsolML/L,CStarAt0,CStarTr0,Hex name,Start Latitude,Start Longitude,Start Time,Cast
0,0,2,Sep 08 2014 12:07:58,1509.032,1489.167,62.10702,-31.38174,3.6550,3.6546,3.286371,...,34.9492,6.3251,7.30825,0.0862,97.8697 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
1,1,4,Sep 08 2014 12:08:10,1509.518,1489.645,62.10702,-31.38176,3.6548,3.6544,3.286384,...,34.9493,6.3326,7.30829,0.0865,97.8614 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
2,2,6,Sep 08 2014 12:08:22,1509.285,1489.415,62.10702,-31.38176,3.6547,3.6546,3.286351,...,34.9492,6.3246,7.30832,0.0871,97.8468 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
3,3,8,Sep 08 2014 12:08:30,1509.060,1489.195,62.10702,-31.38176,3.6547,3.6547,3.286340,...,34.9492,6.2751,7.30832,0.0868,97.8543 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
4,4,10,Sep 08 2014 12:08:38,1509.240,1489.371,62.10702,-31.38174,3.6544,3.6543,3.286338,...,34.9491,6.2838,7.30835,0.0868,97.8535 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
5,5,12,Sep 08 2014 12:08:45,1509.079,1489.213,62.10702,-31.38176,3.6544,3.6543,3.286332,...,34.9493,6.2567,7.30835,0.0870,97.8493 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
6,6,13,Sep 08 2014 12:08:54,1509.051,1489.186,62.10702,-31.38174,3.6548,3.6544,3.286355,...,34.9492,6.3015,7.30829,0.0880,97.8256 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
7,7,17,Sep 08 2014 12:09:02,1509.341,1489.470,62.10702,-31.38176,3.6547,3.6542,3.286362,...,34.9493,6.3135,7.30831,0.0868,97.8548 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
8,8,18,Sep 08 2014 12:09:09,1509.254,1489.385,62.10702,-31.38176,3.6547,3.6542,3.286370,...,34.9493,6.2638,7.30830,0.0876,97.8353 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
9,9,20,Sep 08 2014 12:09:18,1509.218,1489.350,62.10702,-31.38176,3.6552,3.6548,3.286397,...,34.9493,6.3244,7.30823,0.0870,97.8490 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1


In [64]:
df.columns.values

array(['Bottle Position', 'C0S/m', 'C1S/m', 'CStarAt0', 'CStarTr0',
       'Cast', 'Cruise ID', 'Date Time', 'DepSM', 'Hex name', 'Latitude',
       'Longitude', 'OxsolML/L', 'PrDM', 'Sal00', 'Sal11', 'Sbeox0ML/L',
       'Start Latitude', 'Start Longitude', 'Start Time', 'T090C',
       'T190C', 'Unnamed: 0'], dtype=object)

In [8]:
# Parse the data for the start_time
def parse_header(header):
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Hex name':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise ID':cruise_id})
        else:
            pass
    
    return hdr
        

In [ ]:
sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == 'prdm') == True]['Full Name']

In [ ]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower())

In [85]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [86]:
df

,Unnamed: 0,Bottle Position,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],...,"Salinity, Practical, 2 [PSU]","Oxygen, SBE 43 [ml/l]","Oxygen Saturation, Garcia & Gordon [ml/l]","Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Hex name,Start Latitude,Start Longitude,Start Time,Cast
0,0,2,Sep 08 2014 12:07:58,1509.032,1489.167,62.10702,-31.38174,3.6550,3.6546,3.286371,...,34.9492,6.3251,7.30825,0.0862,97.8697 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
1,1,4,Sep 08 2014 12:08:10,1509.518,1489.645,62.10702,-31.38176,3.6548,3.6544,3.286384,...,34.9493,6.3326,7.30829,0.0865,97.8614 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
2,2,6,Sep 08 2014 12:08:22,1509.285,1489.415,62.10702,-31.38176,3.6547,3.6546,3.286351,...,34.9492,6.3246,7.30832,0.0871,97.8468 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
3,3,8,Sep 08 2014 12:08:30,1509.060,1489.195,62.10702,-31.38176,3.6547,3.6547,3.286340,...,34.9492,6.2751,7.30832,0.0868,97.8543 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
4,4,10,Sep 08 2014 12:08:38,1509.240,1489.371,62.10702,-31.38174,3.6544,3.6543,3.286338,...,34.9491,6.2838,7.30835,0.0868,97.8535 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
5,5,12,Sep 08 2014 12:08:45,1509.079,1489.213,62.10702,-31.38176,3.6544,3.6543,3.286332,...,34.9493,6.2567,7.30835,0.0870,97.8493 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
6,6,13,Sep 08 2014 12:08:54,1509.051,1489.186,62.10702,-31.38174,3.6548,3.6544,3.286355,...,34.9492,6.3015,7.30829,0.0880,97.8256 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
7,7,17,Sep 08 2014 12:09:02,1509.341,1489.470,62.10702,-31.38176,3.6547,3.6542,3.286362,...,34.9493,6.3135,7.30831,0.0868,97.8548 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
8,8,18,Sep 08 2014 12:09:09,1509.254,1489.385,62.10702,-31.38176,3.6547,3.6542,3.286370,...,34.9493,6.2638,7.30830,0.0876,97.8353 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
9,9,20,Sep 08 2014 12:09:18,1509.218,1489.350,62.10702,-31.38176,3.6552,3.6548,3.286397,...,34.9493,6.3244,7.30823,0.0870,97.8490 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1


In [90]:
df.shape

(98, 21)

In [88]:
df.drop(columns='Unnamed: 0',inplace=True)

In [89]:
df

,Bottle Position,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],"Conductivity, 2 [S/m]",...,"Salinity, Practical, 2 [PSU]","Oxygen, SBE 43 [ml/l]","Oxygen Saturation, Garcia & Gordon [ml/l]","Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Hex name,Start Latitude,Start Longitude,Start Time,Cast
0,2,Sep 08 2014 12:07:58,1509.032,1489.167,62.10702,-31.38174,3.6550,3.6546,3.286371,3.286591,...,34.9492,6.3251,7.30825,0.0862,97.8697 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
1,4,Sep 08 2014 12:08:10,1509.518,1489.645,62.10702,-31.38176,3.6548,3.6544,3.286384,3.286602,...,34.9493,6.3326,7.30829,0.0865,97.8614 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
2,6,Sep 08 2014 12:08:22,1509.285,1489.415,62.10702,-31.38176,3.6547,3.6546,3.286351,3.286600,...,34.9492,6.3246,7.30832,0.0871,97.8468 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
3,8,Sep 08 2014 12:08:30,1509.060,1489.195,62.10702,-31.38176,3.6547,3.6547,3.286340,3.286596,...,34.9492,6.2751,7.30832,0.0868,97.8543 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
4,10,Sep 08 2014 12:08:38,1509.240,1489.371,62.10702,-31.38174,3.6544,3.6543,3.286338,3.286569,...,34.9491,6.2838,7.30835,0.0868,97.8535 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
5,12,Sep 08 2014 12:08:45,1509.079,1489.213,62.10702,-31.38176,3.6544,3.6543,3.286332,3.286574,...,34.9493,6.2567,7.30835,0.0870,97.8493 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
6,13,Sep 08 2014 12:08:54,1509.051,1489.186,62.10702,-31.38174,3.6548,3.6544,3.286355,3.286575,...,34.9492,6.3015,7.30829,0.0880,97.8256 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
7,17,Sep 08 2014 12:09:02,1509.341,1489.470,62.10702,-31.38176,3.6547,3.6542,3.286362,3.286573,...,34.9493,6.3135,7.30831,0.0868,97.8548 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
8,18,Sep 08 2014 12:09:09,1509.254,1489.385,62.10702,-31.38176,3.6547,3.6542,3.286370,3.286571,...,34.9493,6.2638,7.30830,0.0876,97.8353 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1
9,20,Sep 08 2014 12:09:18,1509.218,1489.350,62.10702,-31.38176,3.6552,3.6548,3.286397,3.286621,...,34.9493,6.3244,7.30823,0.0870,97.8490 (avg),C:\Data\ctd\KN22104001.hex,62 06.42 N,031 22.90 W,2014-09-08T11:39:06Z,1


In [91]:
df.to_csv(dirpath+'Irminger-1_Summary.csv')